In [6]:
from Data_interpret.GroundTruth import ReadMat
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import sys

### Gather State Trajectories

In [7]:
# Loading DATA
train_file = "../Raw_data/8_8_2019 to 8_12_2019/2019-08-08-19-16-02_5sec.mat"
train_file2 = "../Raw_data/8_8_2019 to 8_12_2019/2019-08-09-13-01-42_10sec.mat"
train_data = ReadMat(train_file)
train_data2 = ReadMat(train_file2)
# train_data = ReadMat(train_file2)
test_file = "../Raw_data/8_8_2019 to 8_12_2019/2019-08-09-12-49-56_7sec.mat"
test_data = ReadMat(test_file)

# SAMPLING DATA
sample_size = 100  # sampling of raw data
train_state_traj1, train_action_traj1 = train_data.sampled_mocap_data(sample_size)  # state X time, number of actions X time
train_state_traj2, train_action_traj2 = train_data2.sampled_mocap_data(sample_size)  # state X time, number of actions X time
test_state_traj, test_action_traj = test_data.sampled_mocap_data(sample_size)

In [19]:
test_state_traj.shape

(9, 756)

In [9]:
# define some relevant parameters
dt = 0.01 # sampling time
nstates = 9 # number of states
nctrl = 3 # number of control inputs

### Define Some Helper Functions

In [10]:
# define the Radial Basis Function that will aid in lifting the states
def rbf(X,C,rbf_type):
    
    Cbig = C ; Y = np.zeros([C.shape[1],X.shape[1]])
    
    for i in range(Cbig.shape[1]):
        
        C = np.asarray([Cbig[:,i]]).T
        C = np.tile(C,X.shape[1])
        r_squared = np.sum((X-C)**2,axis = 0)
        
        r_squared = np.reshape(r_squared,(1,len(r_squared)))
        y = r_squared*np.log(np.sqrt(r_squared))
        
        Y[i,:] = y
    
    return Y

In [15]:
Cy = np.ones([9,1]) # Output matrix: y = Cy*x
nD = 1 # Number of delays
ny = Cy.shape[0] # Number of outputs

# Delay-embedded "state" 
# zeta_k = [y_{k} ; u_{k-1} ; y_{k-1} ... u_{k-nd} ; y_{k-nd} ]
n_zeta = (nD+1)*ny + nD*nctrl # dimension of the delay-embedded "state"



In [16]:
n_zeta

21